## free-marketing-watch
Search social media for mentions of brands and collect the comments/tweets/etc.
Count mentions of each and perform sentiment analysis on the strings.

In [1]:
import praw
import pandas as pd
from secrets import *
from pathlib import Path
from brands import fashion
import re

In [2]:
reddit = praw.Reddit(client_id=client_id,
               client_secret=client_secret,
               user_agent=user_agent)

Now to get the comments data, put it in a dataframe, and clean the data to get what we want.

In [3]:
def create_comments_df(subreddit_):
    """Returns a pandas df with the information about comments from this year.

    Inputs
    -----
    str: subreddit to be searched.
    Return
    ------
    Pandas dataframe with all the data from the praw object.m
    """
    subreddit = reddit.subreddit(subreddit_)
    submission_list = subreddit.top(
        time_filter="year", limit=1000
    )  # generator of submissions in the subreddit
    comment_list = []
    for submission in submission_list:
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            comment_list.append(comment)

    df = pd.DataFrame([vars(comment) for comment in comment_list])
    df2 = df.loc[:,['link_id','id','score','body']]
    df2['Subreddit'] = subreddit_
    return df2


In [4]:
def brand_check(df,brandlist):
    """Checks comment body against a list of brands to see if it mentions any.
       Adds what brand was found if any in the brands column.
       
       Inputs
       ------
       Dataframe you will search over and a list of brands in a separate file. 
       Return
       ------
       Dataframe with column indicating what brand was found in the values.
       """

    df2 = df.body.str.extractall(pat=brandlist)
    df2.dropna(axis=0,how='all',inplace=True)
    return df2

### This takes a long time, probably around 30 minutes per 100,000 comments.

In [5]:
df = create_comments_df('femalefashionadvice')
df

,link_id,id,score,body,Subreddit
0,t3_ggpm9u,fq4m7up,1,We've received enough reports that this belong...,femalefashionadvice
1,t3_ggpm9u,fq3ejsm,4019,I'm sure I'll be downvoted for this but... whi...,femalefashionadvice
2,t3_ggpm9u,fq3f8cq,1333,I agree; there needs to be more flexibility in...,femalefashionadvice
3,t3_ggpm9u,fq43dqk,356,I completely agree. I waited 8 hours for a pos...,femalefashionadvice
4,t3_ggpm9u,fq3llwt,627,"Yeah, I think the sub is too strict on moderat...",femalefashionadvice
...,...,...,...,...,...
147337,t3_g04l8d,fn9viaz,2,"If SoWa gets to happen this year, sometimes th...",femalefashionadvice
147338,t3_g04l8d,fnfssbp,1,this gives me hope!! what style did you get??,femalefashionadvice
147339,t3_g04l8d,fnat7hw,1,They have a few. Do you these could be daily ...,femalefashionadvice
147340,t3_g04l8d,fna39ga,2,"Great to note, thanks for the tip!",femalefashionadvice


Run to export the df to csv. Careful about overwriting. Use the mode = 'a' line to add to an existing file.


In [6]:
p = Path.cwd() / 'data' / 'commentdf.csv'
#df.to_csv(path_or_buf = p)
df.to_csv(path_or_buf = p, mode = 'a', header=False)

In [5]:
p = Path.cwd() / 'data' / 'commentdf.csv'
df = pd.read_csv(p)

In [6]:
#df['body'] = df['body'].str.lower()
df = df.drop(columns =['Unnamed: 0'])

In [10]:
df2 = brand_check(df,fashion)

In [23]:
pd.set_option("display.max_rows",50, "display.max_columns", None)
df2

0    1    2    3    4    5    6    7    8       9     10   11  \
      match                                                                   
4     4383   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN   
      4462   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  J Crew   NaN  NaN   
29    40     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  J.Crew   NaN  NaN   
43    0      NaN  Gap  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN   
164   39     NaN  NaN  H&M  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN   
...          ...  ...  ...  ...  ...  ...  ...  ...  ...     ...   ...  ...   
93785 230    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  Zara  NaN   
      355    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  Zara  NaN   
93789 385    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  Zara  NaN   
93793 225    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  Zara  NaN   
93797 98     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  Zara  NaN   

               12  
      match        
4     4383   Vans  
      4462    NaN  
29    40      NaN  
43    0       NaN  
164   39      NaN  
...           ...  
93785 230     NaN  
      355     NaN  
93789 385     NaN  
93793 225     NaN  
93797 98      NaN  

[6248 rows x 13 columns]

In [29]:
df2[6].value_counts()

Apple    124
Name: 6, dtype: int64